In [2]:
import os
import pdfplumber
import pandas as pd
from io import StringIO

data_folder = os.path.abspath('../papers/')
print("data_folder, ", data_folder)
pdf_path = os.path.join(data_folder, 'ASO_rna_seq_EFO21/rna_seq.pdf')

capture = False

table_to_lines = dict()
tables = range(3, 12)

with pdfplumber.open(pdf_path) as pdf:
    for table_num in tables:
        table_lines = []
        for page in pdf.pages:
            text = page.extract_text()
            for line in text.splitlines():
                if capture and f"Supplementary Table {table_num} lists" in line:
                    capture = False
                    break
                if f"Supplementary Table {table_num}" in line:
                    capture = True
                    continue

                if capture and line.strip():
                    table_lines.append(line.strip())
        table_to_lines[table_num] = table_lines

print(len(table_to_lines))
print(table_to_lines)


data_folder,  /mnt/c/Users/Din/Desktop/IGEM_repo/ASOdesign/papers
9
{3: ['ASO ENSEMBL_Transcript_ID ENSEMBL_Gene_ID mean_expression log (FoldChange) padj', '2', '6 ENST00000219368 ENSG00000103089 30 0.63 0.46957', '6 ENST00000221957 ENSG00000105355 1850 -0.38 2.39E-05', '6 ENST00000253048 ENSG00000130749 2992 0.01 0.94192', '6 ENST00000255078 ENSG00000132740 1263 -0.02 0.91409', '6 ENST00000256996 ENSG00000134574 2160 -0.01 0.96537', '6 ENST00000260506 ENSG00000137942 1475 -0.17 0.12111', '6 ENST00000261637 ENSG00000120800 2905 -0.15 0.10374', '6 ENST00000263579 ENSG00000110063 923 0.10 0.52074', '6 ENST00000264313 ENSG00000109171 1564 0.00 0.99015', '6 ENST00000265351 ENSG00000124571 3026 -0.02 0.86196', '6 ENST00000267199 ENSG00000139719 347 0.20 0.31417', '6 ENST00000273398 ENSG00000114573 4484 -0.21 0.00675', '6 ENST00000281141 ENSG00000151465 2234 -0.16 0.09505', '6 ENST00000284116 ENSG00000153982 43 -0.49 0.43461', '6 ENST00000289672 ENSG00000158683 3 0.87 NA', '6 ENST00000294053

In [3]:
dataframes = []
header = 'ASO ENSEMBL_Transcript_ID ENSEMBL_Gene_ID mean_expression log2(FoldChange) padj'

for table_num, table_line in table_to_lines.items():

    if not table_line[0].startswith('ASO'):
        table_line.insert(0, header)
    else:
        table_line[0] = header

    if table_line[1] == '2':
        table_line.pop(1)

    table_text = '\n'.join(table_line)
    data_io = StringIO(table_text)
    df = pd.read_csv(data_io, delim_whitespace=True)
    dataframes.append(df)

concatenated_df = pd.concat(dataframes, ignore_index=True)

print(concatenated_df)

      ASO ENSEMBL_Transcript_ID  ENSEMBL_Gene_ID  mean_expression  \
0       6       ENST00000219368  ENSG00000103089               30   
1       6       ENST00000221957  ENSG00000105355             1850   
2       6       ENST00000253048  ENSG00000130749             2992   
3       6       ENST00000255078  ENSG00000132740             1263   
4       6       ENST00000256996  ENSG00000134574             2160   
...   ...                   ...              ...              ...   
8139   52       ENST00000674197  ENSG00000165868              705   
8140   52       ENST00000674214  ENSG00000168259             2251   
8141   52       ENST00000674284  ENSG00000120458              228   
8142   52       ENST00000674370  ENSG00000152133              551   
8143   52       ENST00000674405  ENSG00000140263             2215   

      log2(FoldChange)      padj  
0                 0.63   0.46957  
1                -0.38  0.000024  
2                 0.01   0.94192  
3                -0.02   0.9140

/tmp/ipykernel_496/3441367560.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(data_io, delim_whitespace=True)
/tmp/ipykernel_496/3441367560.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(data_io, delim_whitespace=True)
/tmp/ipykernel_496/3441367560.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(data_io, delim_whitespace=True)
/tmp/ipykernel_496/3441367560.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(data_io, delim_whitespace=True)
/tmp/ipykernel_496/3441367560.py:16: FutureWarning: The 'delim_whitespace' keywo

In [4]:
paper_path = os.path.join(data_folder, 'ASO_rna_seq_EFO21')
concatenated_df.to_csv(os.path.join(paper_path,'parsed', 'rna_seq.csv'), index=False)